# 类视图
- 在写视图的时候，django除了使用函数作为视图，也可以使用类作为视图，使用类视图可以使用类的一些特性，比如继承等

# View
- 导入：django.views.generic.base.View
- 主要类视图，所有的类视图都是继承自它。想写自己的类视图，也可以继承他。然后再根据当前请求的method，来实现不同的方法。比如这个视图只能使用get请求方法来请求，那么就可在这个类中定义get(self, request, *args, **kwargs)方法。以此类推，如果只需要实现post方法，那么只需要在类中实现post(self, request, *args, **kwargs)
- 1.类视图写完后，要在urls.py中进行映射，映射的时候需要调用view的类方法as_view()来进行转换
    - 示例：
            path('view/', index_views.indexView.as_view(), name='indexView'),
- 2.除了get方法，view还支持以下方法： get, post, put, patch, delete, head, options, trace
- 3.如果用户访问了类视图中没有定义的方法，比如你的类视图只支持get方法，而别人是采用post方法请求，那么这个请求就会转发给http_method_not_allowed(request, *args, **kwargs)
    - 示例：
            class InfoView(View):
                def get(self, request, id, *args, **kwargs):
                    print("你的身份id是%s" % id)
                    return HttpResponse("获取信息成功")

                def http_method_not_allowed(self, request, *args, **kwargs):
                    return HttpResponse("请求方法不允许")
                # 该方法表示如果请求的方法不在允许的方法内，就会返回该结果

- 4.无论是get请求还是post请求，都会走dispatch(request, *args, **kwargs)方法。如果实现了这个方法，将能够对所有的请求都处理到。也就是说，该方法是在请求方法之前就进行的
    - 示例：
            class InfoView(View):
                def get(self, request, id, *args, **kwargs):
                    print("你的身份id是%s" % id)
                    return HttpResponse("获取信息成功")

                def dispatch(self, request, *args, **kwargs):
                    print("dispatch执行了")
                    return super(InfoView, self).dispatch(request, *args, **kwargs)
                # 这里返回的是父类的dispatch的方法 ，如果不调用它，后面就不会分发了

                def http_method_not_allowed(self, request, *args, **kwargs):
                    return HttpResponse("请求方法不允许")
                # 该方法表示如果请求的方法不在允许的方法内，就会返回该结果

# TemplateView
- 导入：django.views.generic.base.Template
- 这个视图是专门用来返回模板的，在这个类中，有两个属性时经常需要用到的，一个是template_name, 这个属性时用来存储模板的路径，templateView会自动的渲染这个变量指向的模板。另外一个是get_context_data,这个方法是用来返回上下文数据的，也就是给模板传递参数的

- 1.在urls.py中映射如下：
    - from django.views.generic.base import TemplateView
    - path('template/', TemplateView.as_view(template_name="two.html")),
    - 这样就不用再写视图函数了，如果不传参的话，推荐使用这种方式
    
- 2.如果要传递参数
    - 示例：
            class MytemplateView(TemplateView):
                template_name = "one.html"

                def get_context_data(self, **kwargs):
                    context = {"address": "成都市武侯区"} #address就会被传进去
                    return context

# ListView
- 在网站开发中，经常出现需要列出某个表中的一些数据作为列表展示出来，比如文章列表。在django中可以用ListView帮我们快速实现这种需求

- 示例代码
        from .models import Person
        from django.http import HttpResponse
        from django.views.generic import ListView

        def add_personInfo(request):
            people = [Person(name="特工{:0>2}号".format(d), age=25, school="特种兵训练营") for d in range(1,101)]
            Person.objects.bulk_create(people)
            return HttpResponse("构建数据成功")


        class PersonListView(ListView):
            model = Person
            template_name = 'personInfo.html'
            context_object_name = 'people'
            paginate_by = 10
            ordering = 'enter_time'
            page_kwarg = 'p'

            def get_context_data(self, **kwargs):
                context = super(PersonListView, self).get_context_data()
                #这个方法最好继承父类的方法，否则容易出错
                context['name'] = 'taotao'
                context['age'] = 23
                context['school'] = '四川大学'
                print("="*59)
                print(context)
                print("="*50)
                return context

            def get_queryset(self):
                return Person.objects.filter(id__gt=55)
            # 这个方法可以对得到的结构进行筛选
- 对以上代码的解释
    - 1.首先PersonListView是继承自ListView
    - 2.model:重写model类属性，指定这个列表是属于哪个模型的
    - 3.template_name：指定这个列表的模板
    - 4.paginate_by：指定这个列表一页中展示多少条数据
    - 5.context_object_name：指定这个列表模型在模板中的参数名称
    - 6.ordering：指定这个列表的排序方式
    - 7.page_kwarg：用来获取第几页的页数的参数名称，默认是page
    - 8.get_context_data：获取上下文的数据
    - 9.get_queryset：如果在提取数据的时候，并不是要把所有数据都返回，那么就可以用这个方法进行过滤

# Paginator和Page类
- Paginator和page类都是来做分页的。它们在django的路径为django.core.paginator.Paginato和django.core.paginator.Page

## Paginator常用属性和方法
- 1.count：总共有多少条数据
- 2.num_page:总共有多少页
- 3.page_range：页面的页数区间

## Page常用属性和方法
- 1.has_next：时候还有下一页
- 2.has_previous：是否还有上一页
- 3.next_page_number：下一页的页码
- 4.previous_page_number
- 5.number：当前页
- 6.start_index：当前这一页的第一条数据的索引值
- 7.end_index：当前这一页的最后一条数据的索引值



# 利用bootstrap自动实现页码效果
- 首先进入https://v3.bootcss.com/getting-started/  ，然后将<!-- 最新版本的 Bootstrap 核心 CSS 文件 -->的css链接复制到自己的模板文件中（即html文件中）


# 手动实现通用分页算法
- 见如下代码

In [ ]:
# ListView视图
class PersonListView(ListView):
    model = Person
    template_name = 'new_personInfo.html'
    context_object_name = 'people'
    paginate_by = 10
    ordering = 'enter_time'
    page_kwarg = 'p'

    def get_context_data(self, **kwargs):
        context = super(PersonListView, self).get_context_data()
        context['name'] = 'taotao'
        context['age'] = 23
        context['school'] = '四川大学'
        paginator = context.get("paginator")
        page_obj = context.get("page_obj")
        paginator_data = self.get_pagination_data(paginator, page_obj)
        context.update(paginator_data)
        # print(context)  #paginator和page就包含在里面，分别是paginator, page_obj
        # print(paginator.count, paginator.num_pages, paginator.page_range)
        # print(page_obj.number, page_obj.has_next(), page_obj.has_previous())
        return context

    # 这是用page和paginator实现分页的核心
    def get_pagination_data(self, paginator, page_obj, around_count=2): 
        current_page = page_obj.number
        num_pages = paginator.num_pages
        left_has_dot = False
        right_has_dot = False

        if current_page <= around_count+2:
            left_pages = range(1, current_page)
        else:
            left_has_dot = True
            left_pages = range(current_page-around_count, current_page)

        if current_page >= num_pages - around_count - 1:
            right_pages = range(current_page+1, num_pages+1)
        else:
            right_has_dot = True
            right_pages = range(current_page+1, current_page+around_count+1)
        return {
            'left_pages': left_pages,
            'right_pages': right_pages,
            'current_page': current_page,
            'left_has_dot': left_has_dot,
            'right_has_dot': right_has_dot,
            'num_pages': num_pages
        }


    # def get_queryset(self):
    #     return Person.objects.filter(id__gt=55)
    # 这个方法可以对得到的结构进行筛选

# 给类视图添加装饰器
- 开发中，有时候需要给一些视图添加装饰器，如果用视图函数非常简单，只要在函数的上面写上装饰器就可以了，但是如果要给类添加装饰器，可以通过以下两种方式来完成。
    - 装饰在dispatch上，直接装饰在整个类上


## 装饰dispatch方法

In [ ]:
# 给dispatch装饰的示例代码，views.py文件的代码
# 给类视图添加装饰器
# from django.utils.decorators import method_decorator
# 内置装饰器，python提供的用来装饰方法的，比较安全。建议在装饰类中的方法时使用它
def login(request):
    return HttpResponse("欢迎来到注册页面")

def login_required(func):
    def wrapper(request, *args, **kwargs):
        username = request.GET.get("username")
        if username:
            return func(request, *args, **kwargs)
        else:
            return redirect(reverse("listView_app:login"))
    return wrapper

class ProfileView(View):
    def get(self, request, *args, **kwargs):
        return HttpResponse("个人中心界面")

    @method_decorator(login_required)
    def dispatch(self, request, *args, **kwargs):
        return super(ProfileView, self).dispatch(request, *args, **kwargs)

In [ ]:
# urls.py文件的代码
from django.urls import path
from . import views as listView

app_name = "listView_app"

urlpatterns = [
    path('add/', listView.add_personInfo, name='info'),
    path('show/', listView.PersonListView.as_view(), name='showlist'),
    path('login/', listView.login, name='login'),
    path('profile/', listView.ProfileView.as_view(), name='profile'),

]

## 直接装饰在整个类上

In [ ]:
# 只需将上面的代码做略微的改动
@method_decorator(login_required, name='dispatch')
class ProfileView(View):
    def get(self, request, *args, **kwargs):
        return HttpResponse("个人中心界面")

    
#     def dispatch(self, request, *args, **kwargs):
#         return super(ProfileView, self).dispatch(request, *args, **kwargs)
# 不要dispatch这个方法了

# 比较
两种方法的话第二种更为简单，不用重写dispatch方法，
@method_decorator(login_required, name='dispatch')，有多个装饰器的话，可以对这个方法传入一个列表